# Semantic Search

In [1]:
!pip install openai
!pip install tiktoken
!pip install streamlit

     ---------------------------------------- 70.1/70.1 kB 3.7 MB/s eta 0:00:00
     ------------------------------------- 323.6/323.6 kB 10.1 MB/s eta 0:00:00
     ---------------------------------------- 56.8/56.8 kB 2.9 MB/s eta 0:00:00
     -------------------------------------- 581.4/581.4 kB 6.1 MB/s eta 0:00:00
     ---------------------------------------- 73.5/73.5 kB 4.0 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 6.5 MB/s eta 0:00:00


In [1]:
import openai
import re
import requests
import sys
import os
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
from openai.embeddings_utils import distances_from_embeddings


openai.api_type = "azure"
openai.api_key = "xxx"
openai.api_base = "xxx"
openai.api_version = "2022-12-01"

# Contracts analyzer and generator

In [20]:
import openai
import re
import requests
import sys
import os
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
from openai.embeddings_utils import distances_from_embeddings



## Key clauses extraction

In [24]:
contract = """
This Contract for Services ("Agreement") is entered into as of [date], by and between Company A ("Company") and Company B ("Service Provider").
1.	Services Provided. Service Provider agrees to provide the following services to Company (the "Services"): The Service Provider agrees to provide consulting services to the Company in the field of marketing, including but not limited to market research, development of a marketing strategy, and implementation of marketing campaigns. The Service Provider shall provide reports and recommendations to the Company based on the results of the market research and the agreed-upon marketing strategy.
2.	Compensation. Company shall pay Service Provider the sum of 1.000.000 (One Million) $ for the Services. Payment shall be made on 15/9/2023.
3.	Term. This Agreement shall commence on 1/5/2023 and continue until 31/12/2023, unless earlier terminated by either party upon 30 days' prior written notice.
4.	Independent Contractor. Service Provider is an independent contractor, and nothing in this Agreement shall be construed as creating an employer-employee relationship, partnership, or joint venture between the parties.
5.	Confidentiality. Service Provider agrees to keep confidential any and all information learned or obtained as a result of providing the Services to Company. Service Provider shall not disclose such information to any third party without Company's prior written consent.
6.	Ownership of Work Product. Service Provider agrees that any and all work product produced in connection with the Services shall be the sole property of Company.
7.	Representations and Warranties. Service Provider represents and warrants that it has the necessary expertise and experience to perform the Services in a professional and workmanlike manner.
8.	Indemnification. Service Provider agrees to indemnify and hold harmless Company, its officers, directors, employees, and agents from and against any and all claims, damages, liabilities, costs, and expenses arising out of or in connection with the Services.
9.	Governing Law. This Agreement shall be governed by and construed in accordance with the laws of Italy without regard to conflicts of laws principles.
10.	Entire Agreement. This Agreement constitutes the entire agreement between the parties and supersedes all prior or contemporaneous negotiations, agreements, representations, and understandings between the parties, whether written or oral.
IN WITNESS WHEREOF, the parties have executed this Agreement as of the date first above written.
[Signature block for Company]
[Signature block for Service Provider]

"""

response = openai.Completion.create(
  engine="test1",
  prompt= contract + "what is the termination clause?",
  temperature=0,
  max_tokens=1968,
  top_p=0.5,
  frequency_penalty=0,
  presence_penalty=0,
  best_of=1,
  stop=None)

print(response["choices"][0]["text"].strip())

The termination clause is clause 3 of the Agreement, which states that the Agreement shall continue until 31/12/2023, unless earlier terminated by either party upon 30 days' prior written notice.


In [25]:
prompt = "<|im_start|>system\n" + contract + "\n<|im_end|>\n"

#print('AI Assistant: ' + prompt + '\n')
while True:
    
    query = input("you:")
    if query == 'q':
        break   
    user_input = "<|im_start|>user\n" + query + "\n<|im_end|>\n<|im_start|>assistant\n"
    prompt+=user_input
    output = openai.Completion.create(
          engine="test1",
          prompt=prompt,
          temperature=0,
          max_tokens=2000,
          top_p=0.95,
          frequency_penalty=0,
          presence_penalty=0,
          stop=["<|im_end|>"])
    print('\n')
    print('AI Assistant: ' + output["choices"][0]["text"].strip() + '\n')
    response = output["choices"][0]["text"].strip() + "\n<|im_end|>\n"
    prompt+=response

you: what is the confidentiality clause?




AI Assistant: The confidentiality clause is a clause in the contract that requires the Service Provider to keep confidential any and all information learned or obtained as a result of providing the Services to Company. The Service Provider shall not disclose such information to any third party without Company's prior written consent.



you: what is the compensation and the due date?




AI Assistant: The compensation is 1.000.000 (One Million) $ and the due date is 15/9/2023.



you: is there an indemnification clause?




AI Assistant: Yes, there is an indemnification clause. Service Provider agrees to indemnify and hold harmless Company, its officers, directors, employees, and agents from and against any and all claims, damages, liabilities, costs, and expenses arising out of or in connection with the Services.



you: q


## Flagging potential issues

In [9]:
contract = """
Sample Employment Agreement
This Agreement is made between ABC Corporation and John Doe, effective as of the date of employment.
1.	Term. This Agreement will last for three years and can be renewed upon mutual agreement.
2.	Duties. John Doe will serve as the Company's Chief Financial Officer and perform assigned responsibilities.
3.	Compensation. John Doe will receive an annual salary of $200,000 and will be eligible for the Company's annual bonus program.
4.	Termination. Either party can terminate this Agreement with written notice. If the Company terminates John Doe without cause, he will receive severance pay equal to 12 months' base salary.
5.	Non-Competition. John Doe agrees not to engage in any competitive business or activity for two years following the termination of this Agreement without prior written consent from the Company.
6.	Governing Law. This Agreement is governed by the laws of the State of Delaware.
7.	Entire Agreement. This Agreement supersedes all prior agreements between the parties.
8.	Amendments. This Agreement cannot be modified except in writing signed by both parties.
9.	Counterparts. This Agreement may be executed in counterparts, each of which shall be deemed an original.

"""

response = openai.Completion.create(
  engine="test1",
  prompt= contract + "Analyze this contract and tell me whether there might be some ambiguity or conflicting terms.",
  temperature=0,
  max_tokens=1968,
  top_p=0.5,
  frequency_penalty=0,
  presence_penalty=0,
  best_of=1,
  stop=None)

print(response["choices"][0]["text"].strip())

There might be some ambiguity in the contract regarding the duties of John Doe. It is not clear what specific responsibilities he will have as the Chief Financial Officer. Additionally, the contract does not state what will happen if either party breaches the agreement. There is also a potential conflict in the terms regarding the severance pay if John Doe is terminated without cause. The contract states that he will receive 12 months' base salary, but it is not clear if this is in addition to the annual salary or if it is included in the annual salary.


In [12]:
prompt = "<|im_start|>system\n" + contract + "\n<|im_end|>\n"

while True:
    
    query = input("you:")
    if query == 'q':
        break   
    user_input = "<|im_start|>user\n" + query + "\n<|im_end|>\n<|im_start|>assistant\n"
    prompt+=user_input
    output = openai.Completion.create(
          engine="test1",
          prompt=prompt,
          temperature=0,
          max_tokens=2000,
          top_p=0.95,
          frequency_penalty=0,
          presence_penalty=0,
          stop=["<|im_end|>"])
    print('\n')
    print('AI Assistant: ' + output["choices"][0]["text"].strip() + '\n')
    response = output["choices"][0]["text"].strip() + "\n<|im_end|>\n"
    prompt+=response

you: Analyze this contract and tell me whether there might be some ambiguity or conflicting terms.




AI Assistant: There might be some ambiguity in the "Duties" section, as it is not clear what exactly John Doe's responsibilities will be. There could also be a conflict in the "Compensation" section if John Doe does not receive the annual bonus that he is expecting.



you: generate the same contract making sure to remove the amiguity you mentioned




AI Assistant: This Agreement is made between ABC Corporation and John Doe, effective as of the date of employment.
1.	Term. This Agreement will last for three years and can be renewed upon mutual agreement.
2.	Duties. John Doe will serve as the Company's Chief Financial Officer and will perform all duties and responsibilities typically associated with this position, as well as any other duties as may be assigned by the Company from time to time.
3.	Compensation. John Doe will receive an annual salary of $200,000 and will be eligible for the Company's annual bonus program.
4.	Termination. Either party can terminate this Agreement with written notice. If the Company terminates John Doe without cause, he will receive severance pay equal to 12 months' base salary.
5.	Non-Competition. John Doe agrees not to engage in any competitive business or activity for two years following the termination of this Agreement without prior written consent from the Company.
6.	Governing Law. This Agreemen

you: explain why there are no ambiguities or conflicting terms now in this new contract




AI Assistant: There are no ambiguities or conflicting terms now because the "Duties" section is more specific about what John Doe's responsibilities will be, and the "Compensation" section makes it clear that John Doe will receive the annual bonus that he is expecting.



you: q


In [16]:
contract = """
This agreement ("Agreement") is made and entered into as of the date of execution by and between ABC Corporation ("ABC") and XYZ Company ("XYZ").

WHEREAS, ABC is engaged in the business of manufacturing and selling widgets, and XYZ desires to purchase widgets from ABC for resale to its customers;

NOW, THEREFORE, in consideration of the premises and mutual covenants herein contained, the parties agree as follows:

Sale of Products. ABC agrees to sell and deliver to XYZ and XYZ agrees to purchase and receive from ABC, widgets as specified in each purchase order issued by XYZ and accepted by ABC.

Price and Payment. The price for the widgets shall be as set forth in the applicable purchase order. Payment shall be due within 30 days of the invoice date.
Non-Competition. ABC agrees not to sell any products to any customers who have purchased products from our competitors within the past six months.
Warranty. ABC warrants that the widgets sold hereunder will conform to the specifications set forth in each purchase order and will be free from defects in material and workmanship for a period of one year from the date of delivery.
Limitation of Liability. ABC's liability for any breach of this Agreement or any warranty provided hereunder shall be limited to the purchase price of the widgets.
Governing Law. This Agreement shall be governed by and construed in accordance with the laws of the State of Delaware.

IN WITNESS WHEREOF, the parties have executed this Agreement as of the date first above written.

ABC Corporation

"""

response = openai.Completion.create(
  engine="test1",
  prompt= contract + "Analyze this contract and tell me whether there are clauses that might violate the antitrust laws. Make sure to highlight those clauses.",
  temperature=0,
  max_tokens=1968,
  top_p=0.5,
  frequency_penalty=0,
  presence_penalty=0,
  best_of=1,
  stop=None)

print(response["choices"][0]["text"].strip())

The clause that states "ABC agrees not to sell any products to any customers who have purchased products from our competitors within the past six months" could violate antitrust laws.


## Providing contracts templates

In [17]:
service_provider = "Supplier ABC"
client = "Company XYZ"
services_description = "installation, configuration, and maintenance of Company's IT infrastructure" 
start_date = "1/9/2023"
duration = "Three (3) years"
payment_terms = f"Within 30 days after receipt of an invoice from {service_provider}"
termination_notice = "30 days"
state = "Italy"


response = openai.Completion.create(
  engine="test1",
  prompt= f"Generate a Service Delivery Agreement with the following elements: Service Provider: {service_provider}, Client: {client}, Description of Services: {services_description}, Start Date: {start_date}, Duration: {duration}, Payment terms: {payment_terms}, Termination notice: {termination_notice}, State or Countries: {state}",
  temperature=0,
  max_tokens=1968,
  top_p=0.5,
  frequency_penalty=0,
  presence_penalty=0,
  best_of=1,
  stop=None)

print(response["choices"][0]["text"].strip())



Service Delivery Agreement

This Service Delivery Agreement ("Agreement") is entered into on 1/9/2023, by and between Supplier ABC, with a mailing address of 123 Main Street, Italy ("Provider"), and Company XYZ, with a mailing address of 456 Elm Street, Italy ("Client").

Description of Services

Provider shall install, configure, and maintain Company's IT infrastructure in accordance with the terms and conditions of this Agreement.

Start Date

This Agreement shall commence on 1/9/2023.

Duration

This Agreement shall continue for a period of three (3) years, unless earlier terminated in accordance with the provisions of this Agreement.

Payment Terms

Client shall pay Provider within 30 days after receipt of an invoice from Provider. All invoices shall be sent to Client's attention at 456 Elm Street, Italy.

Termination Notice

Either party may terminate this Agreement upon 30 days' written notice to the other party.

State or Countries

This Agreement shall be governed by the laws o

## Analyzing language

In [18]:
contract = """
This Carbon Capture and Storage Agreement ("Agreement") is made and entered into on 19/03/2023 by and between Company A, a corporation organized under the laws of France, 
and Company B, a corporation organized under the laws of Italy. The parties agree to collaborate on a CCS project to store 50 tons of CO2 in a saline aquifer located in Southern 
France using enhanced oil recovery (EOR) techniques. The parties will jointly design, construct, and operate the CCS facility, and will share the costs of the project in accordance 
with the proportion of their respective equity interests. Company A will provide the CO2 capture, compression, and transportation equipment, while Company B will provide the storage 
site and injection equipment. Both parties agree to comply with all applicable laws and regulations related to CCS, including the Environmental Protection Agency's (EPA) 
regulations on Underground Injection Control (UIC) Class VI wells.
"""

prompt = "<|im_start|>system\n" + contract + "\n<|im_end|>\n"

#print('AI Assistant: ' + prompt + '\n')
while True:
    
    query = input("you:")
    if query == 'q':
        break   
    user_input = "<|im_start|>user\n" + query + "\n<|im_end|>\n<|im_start|>assistant\n"
    prompt+=user_input
    output = openai.Completion.create(
          engine="test1",
          prompt=prompt,
          temperature=0,
          max_tokens=2000,
          top_p=0.95,
          frequency_penalty=0,
          presence_penalty=0,
          stop=["<|im_end|>"])
    print('\n')
    print('AI Assistant: ' + output["choices"][0]["text"].strip() + '\n')
    response = output["choices"][0]["text"].strip() + "\n<|im_end|>\n"
    prompt+=response

you: what is EPA?




AI Assistant: The Environmental Protection Agency (EPA) is a federal agency responsible for protecting human health and the environment. The EPA's regulations on Underground Injection Control (UIC) Class VI wells govern the construction and operation of CCS facilities.



you: what is the link of EPA with the above contract?




AI Assistant: The EPA's regulations on Underground Injection Control (UIC) Class VI wells govern the construction and operation of CCS facilities.



you: q


# Call Center analytics

In [ ]:
import openai
import re
import requests
import sys
import os
import pandas as pd
import numpy as np



In [ ]:
transcript = "Operator: Good morning, thank you for calling the auto insurance company, my name is John, how can I assist you today?\nCustomer: Yes, hi, I just noticed a dent on the side of my car and I have no idea how it got there. There were no witnesses around and I'm really frustrated.\nOperator: I'm sorry to hear that, I understand how frustrating it can be. Can you please provide me with your name and policy number so I can look up your account?\nCustomer: Yes, I’m Mario Rossi and the policy number is 123456.\nOperator: Thank you Mr. Rossi, let me take a look. I see that you've called earlier today, was there an issue with that call?\nCustomer: Yes, I was on hold for over an hour and the issue was not resolved. I'm really not happy about it.\nOperator: I'm sorry about that, let me assure you that we value your time and we'll do our best to assist you today. As for the dent on your car, I'd like to inform you that our policy does cover accidental damage like this. I can help you file a claim and connect you with one of our trusted repair shops in your area. Would you like me to proceed with that?\nCustomer: Yes, please. That would be great.\nOperator: Thank you for your cooperation. I'm now processing your claim and I'll be sending you an email with the next steps to follow. Please let me know if you have any other questions or concerns.\nCustomer: Thank you, I appreciate your help.\nOperator: You're welcome. Have a great day!\n\n\n"


response = openai.Completion.create(
  engine="test1",
  prompt= transcript + "Extract the following information from the above text:\n Name and Surname\nReason for calling\n Policy Number\n Resolution \n\n. The ouput format should be JSON",
  temperature=1,
  max_tokens=1968,
  top_p=0.5,
  frequency_penalty=0,
  presence_penalty=0,
  best_of=1,
  stop=None)

print(response["choices"][0]["text"].strip())

In [13]:
response = openai.Completion.create(
  engine="test1",
  prompt= transcript + "What is the initial and final sentiment of the conversation?",
  temperature=1,
  max_tokens=1968,
  top_p=0.5,
  frequency_penalty=0,
  presence_penalty=0,
  best_of=1,
  stop=None)

print(response["choices"][0]["text"].strip())

The customer is initially negative because they are frustrated. They were on hold for a long time and their issue was not resolved.


In [15]:
response = openai.Completion.create(
  engine="test1",
  prompt= transcript + """Extract the following information from the above text:\n Name and Surname\nReason for calling
  \n Policy Number
  \n Resolution
  \n Initial Sentiment
  \n Reason for initial Sentiment
  \n Final sentiment\n Reason for final sentiment\n Contact center improvement \n \n\n. The ouput format should be JSON""",
  temperature=1,
  max_tokens=1968,
  top_p=0.5,
  frequency_penalty=0,
  presence_penalty=0,
  best_of=1,
  stop=None)

print(response["choices"][0]["text"].strip())

as follows:

{
  "Name and Surname": "Mario Rossi",
  "Reason for calling": "To file a claim for accidental damage",
  "Policy Number": "123456",
  "Resolution": "The issue was resolved and the customer was assisted with filing a claim",
  "Initial Sentiment": "Frustration",
  "Reason for initial Sentiment": "The customer was frustrated because they did not know how the damage occurred and there were no witnesses",
  "Final sentiment": "Satisfied",
  "Reason for final sentiment": "The customer was assisted with filing a claim and was given information on the next steps to follow",
  "Contact center improvement": "Reduce wait time for customers"
}


In [24]:
response = openai.Completion.create(
  engine="test1",
  prompt= transcript + """classify the above text in one of the following: reporting issue, 
  claim handling -  and general inquiry.""",
  temperature=1,
  max_tokens=1968,
  top_p=0.5,
  frequency_penalty=0,
  presence_penalty=0,
  best_of=1,
  stop=None)

print(response["choices"][0]["text"].strip())

This is a claim handling issue.


In [28]:
response = openai.Completion.create(
  engine="test1",
  prompt= transcript + 
    """Extract the following information from the above text:\n Name and Surname\nReason for calling
    \n Policy Number\n Resolution\n Initial Sentiment\n Reason for initial Sentiment\n Final sentiment
    \n Reason for final sentiment\n Contact center improvement\n Category (choose among reporting issue, 
    claim handling and general inquiry) \n \n\n. Return the output in JSON.""",
  temperature=1,
  max_tokens=1968,
  top_p=0.5,
  frequency_penalty=0,
  presence_penalty=0,
  best_of=1,
  stop=None)

print(response["choices"][0]["text"].strip())

{
    "name": "Mario Rossi",
    "reason_for_calling": "To report a dent on the side of his car",
    "policy_number": "123456",
    "resolution": "The operator helped file a claim and connect the customer with a trusted repair shop in the area.",
    "initial_sentiment": "Frustration",
    "reason_for_initial_sentiment": "The customer was frustrated because he did not know how the dent got there and there were no witnesses around.",
    "final_sentiment": "Satisfaction",
    "reason_for_final_sentiment": "The customer was satisfied because the operator was able to help him file a claim and connect him with a trusted repair shop.",
    "contact_center_improvement": "The customer would like the contact center to value his time more and not keep him on hold for long periods of time.",
    "category": "Claim Handling"
}


In [31]:
import json
with open('json_data.json', 'w') as outfile:
    outfile.write(response["choices"][0]["text"].strip())

In [32]:
import json

# Opening JSON file
f = open('json_data.json')

# returns JSON object as
# a dictionary
data = json.load(f)
data

{'name': 'Mario Rossi',
 'reason_for_calling': 'To report a dent on the side of his car',
 'policy_number': '123456',
 'resolution': 'The operator helped file a claim and connect the customer with a trusted repair shop in the area.',
 'initial_sentiment': 'Frustration',
 'reason_for_initial_sentiment': 'The customer was frustrated because he did not know how the dent got there and there were no witnesses around.',
 'final_sentiment': 'Satisfaction',
 'reason_for_final_sentiment': 'The customer was satisfied because the operator was able to help him file a claim and connect him with a trusted repair shop.',
 'contact_center_improvement': 'The customer would like the contact center to value his time more and not keep him on hold for long periods of time.',
 'category': 'Claim Handling'}